In [1]:
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting(use_latex='mathjax')

In [2]:
class ReferenceFrame(me.ReferenceFrame):

    def __init__(self, *args, **kwargs):

        kwargs.pop('latexs', None)

        lab = args[0].lower()
        tex = r'\hat{{{}}}_{}'

        super(ReferenceFrame, self).__init__(*args,
                                             latexs=(tex.format(lab, 'x'),
                                                     tex.format(lab, 'y'),
                                                     tex.format(lab, 'z')),
                                             **kwargs)
me.ReferenceFrame = ReferenceFrame

In [3]:
x, y, theta = me.dynamicsymbols('x, y, theta')

N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')

A.orient_axis(N, theta, N.z)

O = me.Point('O')
P = me.Point('P')

P.set_pos(O, x*N.x + y*N.y)

O.set_vel(N, 0)

P.vel(N).express(A)

(sin(θ)⋅ẏ + cos(θ)⋅ẋ) a_x + (-sin(θ)⋅ẋ + cos(θ)⋅ẏ) a_y

In [4]:
fn = P.vel(N).dot(A.y)
fn

-sin(θ)⋅ẋ + cos(θ)⋅ẏ

In [5]:
t = me.dynamicsymbols._t

q1, q2, q3 = me.dynamicsymbols('q1, q2, q3')
la, lb, lc, ln = sm.symbols('l_a, l_b, l_c, l_n')

fhx = la*sm.cos(q1) + lb*sm.cos(q1 + q2) + lc*sm.cos(q1 + q2 + q3) - ln
sm.trigsimp(fhx.diff(t))

-lₐ⋅sin(q₁)⋅q₁̇ - l_b⋅(q₁̇ + q₂̇)⋅sin(q₁ + q₂) - l_c⋅(q₁̇ + q₂̇ + q₃̇)⋅sin(q₁ 
+ q₂ + q₃)

In [6]:
dfdx = fn.coeff(x.diff(t))
dfdy = fn.coeff(y.diff(t))
dfdth = fn.coeff(theta.diff(t))

dfdx, dfdy, dfdth

(-sin(θ), cos(θ), 0)

In [7]:
dfdx.diff(y), dfdy.diff(x)

(0, 0)

In [8]:
dfdx.diff(theta), dfdth.diff(x)

(-cos(θ), 0)

In [9]:
dfdy.diff(theta), dfdth.diff(y)

(-sin(θ), 0)

In [10]:
fnx = fhx.diff(t)
dfdq1 = fnx.diff(q1)
dfdq2 = fnx.diff(q2)
dfdq3 = fnx.diff(q3)

In [11]:
dfdq1.diff(q2) -  dfdq2.diff(q1)

0

In [12]:
dfdq2.diff(q3) - dfdq3.diff(q2)

0

In [13]:
dfdq3.diff(q1) - dfdq1.diff(q3)

0

In [14]:
q1, q2, q3 = me.dynamicsymbols('q1, q2, q3')

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

B.orient_body_fixed(A, (q1, q2, q3), 'ZXY')

A_w_B = B.ang_vel_in(A).simplify()
A_w_B

(-sin(q₃)⋅cos(q₂)⋅q₁̇ + cos(q₃)⋅q₂̇) b_x + (sin(q₂)⋅q₁̇ + q₃̇) b_y + (sin(q₃)⋅
q₂̇ + cos(q₂)⋅cos(q₃)⋅q₁̇) b_z

In [15]:
u1, u2, u3 = me.dynamicsymbols('u1, u2, u3')

t = me.dynamicsymbols._t
qdot = sm.Matrix([q1.diff(t), q2.diff(t), q3.diff(t)])
u = sm.Matrix([u1, u2, u3])

A_w_B = A_w_B.xreplace(dict(zip(qdot, u)))
A_w_B

(-u₁⋅sin(q₃)⋅cos(q₂) + u₂⋅cos(q₃)) b_x + (u₁⋅sin(q₂) + u₃) b_y + (u₁⋅cos(q₂)⋅c
os(q₃) + u₂⋅sin(q₃)) b_z

In [16]:
Yk_plus_zk = qdot
Yk_plus_zk

⎡q₁̇⎤
⎢  ⎥
⎢q₂̇⎥
⎢  ⎥
⎣q₃̇⎦

In [17]:
Yk = Yk_plus_zk.jacobian(qdot)
Yk

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [18]:
qd_zero_repl = dict(zip(qdot, sm.zeros(3, 1)))
qd_zero_repl

{q₁̇: 0, q₂̇: 0, q₃̇: 0}

In [19]:
zk = Yk_plus_zk.xreplace(qd_zero_repl)
zk

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [20]:
sm.Eq(qdot, Yk.LUsolve(u - zk))

⎡q₁̇⎤   ⎡u₁⎤
⎢  ⎥   ⎢  ⎥
⎢q₂̇⎥ = ⎢u₂⎥
⎢  ⎥   ⎢  ⎥
⎣q₃̇⎦   ⎣u₃⎦

In [21]:
A_w_B = B.ang_vel_in(A).simplify()
A_w_B

(-sin(q₃)⋅cos(q₂)⋅q₁̇ + cos(q₃)⋅q₂̇) b_x + (sin(q₂)⋅q₁̇ + q₃̇) b_y + (sin(q₃)⋅
q₂̇ + cos(q₂)⋅cos(q₃)⋅q₁̇) b_z

In [22]:
u1_expr = A_w_B.dot(B.x)
u2_expr = A_w_B.dot(B.y)
u3_expr = A_w_B.dot(B.z)

Yk_plus_zk = sm.Matrix([u1_expr, u2_expr, u3_expr])
Yk_plus_zk

⎡-sin(q₃)⋅cos(q₂)⋅q₁̇ + cos(q₃)⋅q₂̇⎤
⎢                                ⎥
⎢        sin(q₂)⋅q₁̇ + q₃̇         ⎥
⎢                                ⎥
⎣sin(q₃)⋅q₂̇ + cos(q₂)⋅cos(q₃)⋅q₁̇ ⎦

In [23]:
Yk = Yk_plus_zk.jacobian(qdot)
Yk

⎡-sin(q₃)⋅cos(q₂)  cos(q₃)  0⎤
⎢                            ⎥
⎢    sin(q₂)          0     1⎥
⎢                            ⎥
⎣cos(q₂)⋅cos(q₃)   sin(q₃)  0⎦

In [24]:
zk = Yk_plus_zk.xreplace(qd_zero_repl)
zk

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [25]:
sm.Eq(qdot, sm.trigsimp(Yk.LUsolve(u - zk)))

       ⎡        -(u₁⋅sin(q₃) - u₃⋅cos(q₃))          ⎤
       ⎢        ───────────────────────────         ⎥
⎡q₁̇⎤   ⎢                  cos(q₂)                   ⎥
⎢  ⎥   ⎢                                            ⎥
⎢q₂̇⎥ = ⎢      u₁⋅cos(2⋅q₃) + u₁ + u₃⋅sin(2⋅q₃)      ⎥
⎢  ⎥   ⎢      ────────────────────────────────      ⎥
⎣q₃̇⎦   ⎢                 2⋅cos(q₃)                  ⎥
       ⎢                                            ⎥
       ⎣u₁⋅sin(q₃)⋅tan(q₂) + u₂ - u₃⋅cos(q₃)⋅tan(q₂)⎦

In [26]:
A_w_B = B.ang_vel_in(A).express(A).simplify()
A_w_B

(-sin(q₁)⋅cos(q₂)⋅q₃̇ + cos(q₁)⋅q₂̇) a_x + (sin(q₁)⋅q₂̇ + cos(q₁)⋅cos(q₂)⋅q₃̇)
 a_y + (sin(q₂)⋅q₃̇ + q₁̇) a_z

In [27]:
u1_expr = A_w_B.dot(A.x)
u2_expr = A_w_B.dot(A.y)
u3_expr = A_w_B.dot(A.z)

Yk_plus_zk = sm.Matrix([u1_expr, u2_expr, u3_expr])
Yk_plus_zk

⎡-sin(q₁)⋅cos(q₂)⋅q₃̇ + cos(q₁)⋅q₂̇⎤
⎢                                ⎥
⎢sin(q₁)⋅q₂̇ + cos(q₁)⋅cos(q₂)⋅q₃̇ ⎥
⎢                                ⎥
⎣        sin(q₂)⋅q₃̇ + q₁̇         ⎦

In [28]:
Yk = Yk_plus_zk.jacobian(qdot)
Yk

⎡0  cos(q₁)  -sin(q₁)⋅cos(q₂)⎤
⎢                            ⎥
⎢0  sin(q₁)  cos(q₁)⋅cos(q₂) ⎥
⎢                            ⎥
⎣1     0         sin(q₂)     ⎦

In [29]:
zk = Yk_plus_zk.xreplace(qd_zero_repl)
zk

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [30]:
sm.Eq(qdot, sm.trigsimp(Yk.LUsolve(u - zk)))

       ⎡(u₁⋅sin(q₁) - u₂⋅cos(q₁))⋅tan(q₂) + u₃⎤
⎡q₁̇⎤   ⎢                                      ⎥
⎢  ⎥   ⎢       u₁⋅cos(q₁) + u₂⋅sin(q₁)        ⎥
⎢q₂̇⎥ = ⎢                                      ⎥
⎢  ⎥   ⎢     -(u₁⋅sin(q₁) - u₂⋅cos(q₁))       ⎥
⎣q₃̇⎦   ⎢     ───────────────────────────      ⎥
       ⎣               cos(q₂)                ⎦

In [31]:
q1, q2, q3, q4, q5 = me.dynamicsymbols('q1, q2, q3, q4, q5')
l = sm.symbols('l')

In [32]:
N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
C = me.ReferenceFrame('C')

A.orient_axis(N, q3, N.z)
B.orient_axis(A, q4, A.z)
C.orient_axis(A, q5, A.z)

In [33]:
A.ang_vel_in(N)

q₃̇ n_z

In [34]:
B.ang_vel_in(N)

q₄̇ a_z + q₃̇ n_z

In [35]:
C.ang_vel_in(N)

q₅̇ a_z + q₃̇ n_z

In [36]:
O = me.Point('O')
Ao = me.Point('A_o')
Bo = me.Point('B_o')
Co = me.Point('C_o')

Ao.set_pos(O, q1*N.x + q2*N.y)
Bo.set_pos(Ao, l/2*A.x)
Co.set_pos(Ao, -l/2*A.x)

In [37]:
O.set_vel(N, 0)
Ao.vel(N)

q₁̇ n_x + q₂̇ n_y

In [38]:
Bo.v2pt_theory(Ao, N, A)

                  l⋅q₃̇
q₁̇ n_x + q₂̇ n_y + ──── a_y
                   2

In [39]:
Co.v2pt_theory(Ao, N, A)

                  -l⋅q₃̇
q₁̇ n_x + q₂̇ n_y + ────── a_y
                    2

In [40]:
fn = sm.Matrix([Bo.vel(N).dot(B.y),
                Co.vel(N).dot(C.y)])
fn = sm.trigsimp(fn)
fn

⎡ l⋅cos(q₄)⋅q₃̇                                     ⎤
⎢ ──────────── - sin(q₃ + q₄)⋅q₁̇ + cos(q₃ + q₄)⋅q₂̇ ⎥
⎢      2                                           ⎥
⎢                                                  ⎥
⎢  l⋅cos(q₅)⋅q₃̇                                    ⎥
⎢- ──────────── - sin(q₃ + q₅)⋅q₁̇ + cos(q₃ + q₅)⋅q₂̇⎥
⎣       2                                          ⎦

In [41]:
u1, u2, u3, u4, u5 = me.dynamicsymbols('u1, u2, u3, u4, u5')

u_repl = {
    q1.diff(): u1,
    q2.diff(): u2,
    l*q3.diff()/2: u3,
    q4.diff(): u4,
    q5.diff(): u5
}

fn = fn.subs(u_repl)
fn

⎡-u₁⋅sin(q₃ + q₄) + u₂⋅cos(q₃ + q₄) + u₃⋅cos(q₄)⎤
⎢                                               ⎥
⎣-u₁⋅sin(q₃ + q₅) + u₂⋅cos(q₃ + q₅) - u₃⋅cos(q₅)⎦

In [42]:
us = sm.Matrix([u3, u4, u5])
ur = sm.Matrix([u1, u2])

In [43]:
Ar = fn.jacobian(ur)
Ar

⎡-sin(q₃ + q₄)  cos(q₃ + q₄)⎤
⎢                           ⎥
⎣-sin(q₃ + q₅)  cos(q₃ + q₅)⎦

In [44]:
As = fn.jacobian(us)
As

⎡cos(q₄)   0  0⎤
⎢              ⎥
⎣-cos(q₅)  0  0⎦

In [45]:
brs = fn.xreplace(dict(zip([u1, u2, u3, u4, u5], [0, 0, 0, 0, 0])))
brs

⎡0⎤
⎢ ⎥
⎣0⎦

In [46]:
An = Ar.LUsolve(-As)
An = sm.simplify(An)
An

⎡cos(q₃ - q₄ + q₅)   cos(q₃ + q₄ - q₅)                          ⎤
⎢───────────────── + ───────────────── + cos(q₃ + q₄ + q₅)      ⎥
⎢        2                   2                                  ⎥
⎢─────────────────────────────────────────────────────────  0  0⎥
⎢                       sin(q₄ - q₅)                            ⎥
⎢                                                               ⎥
⎢sin(q₃ - q₄ + q₅)   sin(q₃ + q₄ - q₅)                          ⎥
⎢───────────────── + ───────────────── + sin(q₃ + q₄ + q₅)      ⎥
⎢        2                   2                                  ⎥
⎢─────────────────────────────────────────────────────────  0  0⎥
⎣                       sin(q₄ - q₅)                            ⎦

In [47]:
bn = Ar.LUsolve(-brs)
bn

⎡0⎤
⎢ ⎥
⎣0⎦

In [48]:
sm.Eq(ur, An*us + bn)

       ⎡⎛cos(q₃ - q₄ + q₅)   cos(q₃ + q₄ - q₅)                    ⎞   ⎤
       ⎢⎜───────────────── + ───────────────── + cos(q₃ + q₄ + q₅)⎟⋅u₃⎥
       ⎢⎝        2                   2                            ⎠   ⎥
       ⎢──────────────────────────────────────────────────────────────⎥
⎡u₁⎤   ⎢                         sin(q₄ - q₅)                         ⎥
⎢  ⎥ = ⎢                                                              ⎥
⎣u₂⎦   ⎢⎛sin(q₃ - q₄ + q₅)   sin(q₃ + q₄ - q₅)                    ⎞   ⎥
       ⎢⎜───────────────── + ───────────────── + sin(q₃ + q₄ + q₅)⎟⋅u₃⎥
       ⎢⎝        2                   2                            ⎠   ⎥
       ⎢──────────────────────────────────────────────────────────────⎥
       ⎣                         sin(q₄ - q₅)                         ⎦